# Running the Random Projection Embedding

In this notebook we're going to generate graph embeddings using the Random Projection algorithm. We'll then explore those embeddings using Python Data Science tools.

Let's start by importing some libraries:

In [148]:
from neo4j import GraphDatabase
from sklearn.manifold import TSNE

import numpy as np
import altair as alt
import altair_viewer
import pandas as pd

Once we've done that we can initialise the Neo4j driver. 

In [ ]:
driver = GraphDatabase.driver("bolt://localhost", auth=("neo4j", "neo"))

We should have already imported the dataset. We can run the following query to check that the data has been imported:

In [147]:
result = {"label": [], "count": []}
with driver.session(database="foo") as session:
    for row in session.run("CALL db.labels()"):
        label = row["label"]
        query = f"MATCH (:`{label}`) RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["label"].append(label)
        result["count"].append(count)
nodes_df = pd.DataFrame(data=result)
nodes_df.sort_values("count")

result = {"relType": [], "count": []}
with driver.session(database="foo") as session:
    for row in session.run("CALL db.relationshipTypes()"):
        relationship_type = row["relationshipType"]
        query = f"MATCH ()-[:`{relationship_type}`]->() RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["relType"].append(relationship_type)
        result["count"].append(count)
rels_df = pd.DataFrame(data=result)
rels_df.sort_values("count")

display(nodes_df)
display(rels_df)

,label,count
0,Place,894


,relType,count
0,EROAD,2500


We should have 894 `Place` nodes and 2,500 `EROAD` relationships.

Now let's run some embeddings. We're going to run the streaming version of the Random Projection algorithm. We need to define the following config:

* `nodeProjection` - the node labels to use for our projected graph
* `relationshipProjection` - the relationship types to use for our projected graph
* `embeddingSize` - the size of the vector/list of numbers to create for each node
* `maxIterations` - the number of iterations to run

Let's give it a try:

In [15]:
with driver.session(database="foo") as session:
    result = session.run("""
    CALL gds.alpha.randomProjection.stream({
       nodeProjection: "Place",
       relationshipProjection: {
         eroad: {
           type: "EROAD",
           orientation: "UNDIRECTED"
        }
       },
       embeddingSize: 10,
       maxIterations: 1
    })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).name AS place, embedding
    LIMIT 10
    """)
    
    embeddings_df = pd.DataFrame([dict(record) for record in result])
embeddings_df    

,place,embedding
0,Larne,"[0.0, 0.18257418274879456, 0.0, 0.0, -0.547722..."
1,Belfast,"[0.18257418274879456, 0.0, -0.0912870913743972..."
2,Dublin,"[0.0, 0.13693062961101532, 0.13693062961101532..."
3,Wexford,"[0.13693062961101532, 0.41079187393188477, 0.0..."
4,Rosslare,"[0.13693062961101532, 0.27386125922203064, 0.0..."
5,La Coruña,"[0.0, 0.3651483654975891, -0.18257418274879456..."
6,Pontevedra,"[0.0, 0.0, 0.0, -0.5477225184440613, -0.273861..."
7,Valença do Minho,"[-0.27386125922203064, 0.0, 0.2738612592220306..."
8,Porto,"[-0.18257418274879456, 0.18257418274879456, 0...."
9,Aveiro,"[0.0, -0.13693062961101532, 0.1369306296110153..."


So far everything looks good. Let's now store the embeddings in Neo4j, by using the write version of the algorithm:

In [109]:
with driver.session(database="foo") as session:
    result = session.run("""
    CALL gds.alpha.randomProjection.write({
       nodeProjection: "Place",
       relationshipProjection: {
         eroad: {
           type: "EROAD",
           orientation: "UNDIRECTED"
        }
       },
       embeddingSize: 10,
       maxIterations: 1,
       writeProperty: $embeddingProperty
    })
    """, {"embeddingProperty": "embeddingRandomProjection"})
    
    embeddings_df = pd.DataFrame([dict(record) for record in result])
embeddings_df    

,nodeCount,nodePropertiesWritten,createMillis,computeMillis,writeMillis,configuration
0,894,894,10,36,26,"{'maxIterations': 1, 'writeConcurrency': 4, 'n..."


The embeddings will be stored in the `embeddingRandomProjection` property on each node. Let's now build a data frame that contains each place and its embedding so that we can explore them further.

In [134]:
with driver.session(database="foo") as session:
    result = session.run("""
    MATCH (p:Place)
    RETURN p.name AS place, p.embeddingRandomProjection AS embedding, p.country AS country
    """)
    X = pd.DataFrame([dict(record) for record in result])
X    

,place,embedding,country
0,Larne,"[-0.18257418274879456, 0.0, 0.0, 0.36514836549...",GB
1,Belfast,"[-0.09128709137439728, 0.18257418274879456, -0...",GB
2,Dublin,"[0.0, 0.13693062961101532, 0.27386125922203064...",IRL
3,Wexford,"[0.0, -0.13693062961101532, -0.136930629611015...",IRL
4,Rosslare,"[0.13693062961101532, 0.0, 0.41079190373420715...",IRL
...,...,...,...
889,Vólos,"[0.5477225184440613, -0.5477225184440613, 0.0,...",GR
890,Trapani,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.54772251...",I
891,Merzifon,"[-0.5477225184440613, 0.5477225184440613, -0.5...",TR
892,Gíthio,"[0.5477225184440613, 0.0, 0.0, 0.0, 0.0, 0.0, ...",GR


## Visualizing Random Projection embeddings

We can visualize our embeddings with the help of the [t-SNE algorithm](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding). 

t-SNE is a dimensionality reduction technique that can be used to reduce high dimensionality objects to 2 or 3 dimensions so that they can be better visualized. We're going to use it to create a scatterplot of our embeddings.

The following code snippet applies t-SNE to the embeddings and then creates a data frame containing each place, its country, as well as x and y coordinates.

In [135]:
X_embedded = TSNE(n_components=2, random_state=6).fit_transform(list(X.embedding))

places = X.place
df = pd.DataFrame(data = {
    "place": places,
    "country": X.country,
    "x": [value[0] for value in X_embedded],
    "y": [value[1] for value in X_embedded]
})
df

,place,country,x,y
0,Larne,GB,18.039846,20.024580
1,Belfast,GB,-3.358563,-17.995136
2,Dublin,IRL,-2.909632,-7.746000
3,Wexford,IRL,-13.913317,1.407290
4,Rosslare,IRL,-19.017954,12.351611
...,...,...,...,...
889,Vólos,GR,6.698317,7.730943
890,Trapani,I,-8.629001,-13.253885
891,Merzifon,TR,-4.648191,-31.553631
892,Gíthio,GR,-12.572576,-16.032948


We can then use the Altair visualization library to create a scatterplot of these coordinates:

In [138]:
chart = alt.Chart(df).mark_circle(size=60).encode(
    x='x',
    y='y',
    tooltip=['place', 'country']
).properties(width=700, height=400, title="Random Projection Embeddings")
chart.save('randomProjection.json')
chart

alt.Chart(...)

There don't seem to be any clusters of points in our visualization. It's also hard to tell what each point represents without hovering over them individually. We can color each point based on their `country` property with the following code:

In [140]:
chart = alt.Chart(df).mark_circle(size=60).encode(
    x='x',
    y='y',
    color='country',
    tooltip=['place', 'country']
).properties(width=700, height=400, title="Random Projection Embeddings")

chart.save('randomProjection-color.json')

chart

alt.Chart(...)